# Supplementary Figures - Upscaled CROP-seq KO150 screen Euler diagrams

In [1]:
# set correct working directory -> project folder
getwd()
setwd('..')
getwd()

[1] "/home/sreichl/projects/bmdm-stim/workflow/notebooks"

[1] "/home/sreichl/projects/bmdm-stim"

In [2]:
# source the figure theme, libraries and utility functions
source("./src/figure_theme.R")

In [3]:
# for euler plots
library(eulerr)

# configs

In [4]:
# configs
data_path <- file.path('results','KO150')
results_path <- file.path('results','figures','supp_large_screen')

In [5]:
# make directories if not exist
dir.create(results_path, showWarnings = FALSE, recursive = TRUE)

# DEG Euler diagrams of selected mixscape DEA results

In [6]:
selected_kos <- c('Ifnar1','Stat2','Irf9')

In [7]:
conditions <- c('untreated', 'LO28-6h', 'LO28-24h')

In [8]:
dea_results <- list()

for (cond in conditions){
    cond_dea <- data.frame()
    for (ko in selected_kos){
        tmp_dea <- read.csv(file.path(data_path, 'DEA_condition_mixscape', paste0('KO150_DEA_RNA_',cond,'_',ko,'.csv')))
        tmp_dea$ko <- ko
        colnames(tmp_dea)[1] <- 'gene'
        
        if(dim(cond_dea)[1]==0){
            cond_dea <- tmp_dea
        }else{
            cond_dea <- rbind(cond_dea, tmp_dea)
        }
    }
    # filter for stat. sign.
    cond_dea <- cond_dea[cond_dea$p_val_adj<0.05,]
    # add group information
    cond_dea$direction <- lapply(cond_dea$avg_log2FC, function(x) if(x>0)'up' else 'down')
    cond_dea$group <- paste0(cond_dea$ko,'_',cond_dea$direction)
    
    dea_results[[cond]] <- cond_dea
}

In [9]:
dea_sets <- list()

for (cond in conditions){
    cond_sets <- data.frame(matrix(ncol = 0, nrow = length(unique(dea_results[[cond]]$gene))))
    rownames(cond_sets) <- unique(dea_results[[cond]]$gene)
    for (group in unique(dea_results[[cond]]$group)){
        tmp_genes <- unique(dea_results[[cond]][grepl(group, dea_results[[cond]]$group, fixed = TRUE),'gene'])
        cond_sets[tmp_genes, group] <- TRUE
    }
    cond_sets[is.na(cond_sets)] <- FALSE
    
    dea_sets[[cond]] <- cond_sets
}

In [10]:
# no stat. sign. up regulated genes -> add column manually
dea_sets[['untreated']]$Stat2_up <- FALSE

head(dea_sets[['untreated']])

,Ifnar1_down,Ifnar1_up,Stat2_down,Irf9_down,Irf9_up,Stat2_up
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Bst2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE
Ifi27l2a,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE
Ly6e,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE
Irf7,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE
Ifitm3,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE
Isg15,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE


In [11]:
# 'Ifnar1_down','Stat2_down','Irf9_down'
for (cond in conditions){
    svg(filename=file.path(results_path, paste0("DEG_",cond,"_euler_",'Ifnar1_down','Stat2_down','Irf9_down',".svg")), width = 5, height = 5)

    dea_fit <- euler(dea_sets[[cond]][c('Ifnar1_down','Stat2_down','Irf9_down')], shape = "ellipse")
    print(plot(dea_fit, quantities = TRUE))

    dev.off()
}

In [12]:
# 'Ifnar1_up','Stat2_up','Irf9_up'
for (cond in conditions){
    svg(filename=file.path(results_path, paste0("DEG_",cond,"_euler_",'Ifnar1_up','Stat2_up','Irf9_up',".svg")), width = 5, height = 5)

    dea_fit <- euler(dea_sets[[cond]][c('Ifnar1_up','Stat2_up','Irf9_up')], shape = "ellipse")
    print(plot(dea_fit, quantities = TRUE))

    dev.off()
}

Warning message in colSums(id & !empty) == 0 | merged_sets:
“longer object length is not a multiple of shorter object length”


In [13]:
# save data frames
for (cond in conditions){
    write.csv(dea_sets[[cond]], file.path(results_path, paste0("DEG_",cond,"_euler_",'Ifnar1_','Stat2_','Irf9',".csv")))
}